In [3]:
#init spark
!pip install -q pyspark findspark

import findspark
findspark.init()
import pyspark
findspark.find()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("analytics_data").getOrCreate()


In [4]:
path = "../data/bank-additional/bank-additional-full.csv"
raw_df = (
    spark.read
    .option("header", True)
    .option("inferschema", True)
    .option("sep", ";")
    .csv(path)
)

cols = ['contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
target_col = 'y'
df = raw_df.select(*(cols + [target_col]))

df.show(10, truncate=False)

+---------+-----+-----------+--------+--------+-----+--------+---+
|contact  |month|day_of_week|duration|campaign|pdays|previous|y  |
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|may  |mon        |261     |1       |999  |0       |no |
|telephone|may  |mon        |149     |1       |999  |0       |no |
|telephone|may  |mon        |226     |1       |999  |0       |no |
|telephone|may  |mon        |151     |1       |999  |0       |no |
|telephone|may  |mon        |307     |1       |999  |0       |no |
|telephone|may  |mon        |198     |1       |999  |0       |no |
|telephone|may  |mon        |139     |1       |999  |0       |no |
|telephone|may  |mon        |217     |1       |999  |0       |no |
|telephone|may  |mon        |380     |1       |999  |0       |no |
|telephone|may  |mon        |50      |1       |999  |0       |no |
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 10 rows



In [5]:
print("schema df: ")
df.printSchema()
print("__________________________________________")

print("\n So dong df: ")
print(df.count())
print("__________________________________________")


schema df: 
root
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- y: string (nullable = true)

__________________________________________

 So dong df: 
41188
__________________________________________


In [7]:
print("Checking missing values: ")
total = df.count()

for c in df.columns:
    miss = df.filter(F.col(c).isNull()).count()
    print(f"{c:20s} null = {miss}")

Checking missing values: 
contact              null = 0
month                null = 0
day_of_week          null = 0
duration             null = 0
campaign             null = 0
pdays                null = 0
previous             null = 0
y                    null = 0


### * Tỷ lệ phân bố giữa các biến

In [ ]:
from pyspark.sql import functions as F
cat_cols = ['contact', 'month', 'day_of_week', 'y']
for c in cat_cols:
    fre_val = df.groupBy(c).count().orderBy('count', ascending = False)
 
    total = df.count()
    col_expr = ((F.col('count') / total) * 100).cast("double")
    fre_val.withColumn("ratio (%)", F.round(col_expr, 2)).show()


+---------+-----+---------+
|  contact|count|ratio (%)|
+---------+-----+---------+
| cellular|26144|    63.47|
|telephone|15044|    36.53|
+---------+-----+---------+

+-----+-----+---------+
|month|count|ratio (%)|
+-----+-----+---------+
|  may|13769|    33.43|
|  jul| 7174|    17.42|
|  aug| 6178|     15.0|
|  jun| 5318|    12.91|
|  nov| 4101|     9.96|
|  apr| 2632|     6.39|
|  oct|  718|     1.74|
|  sep|  570|     1.38|
|  mar|  546|     1.33|
|  dec|  182|     0.44|
+-----+-----+---------+

+-----------+-----+---------+
|day_of_week|count|ratio (%)|
+-----------+-----+---------+
|        thu| 8623|    20.94|
|        mon| 8514|    20.67|
|        wed| 8134|    19.75|
|        tue| 8090|    19.64|
|        fri| 7827|     19.0|
+-----------+-----+---------+

+---+-----+---------+
|  y|count|ratio (%)|
+---+-----+---------+
| no|36548|    88.73|
|yes| 4640|    11.27|
+---+-----+---------+



### Nhận xét phân tích tần suất các biến phân loại

#### 1. `contact` — Kênh liên hệ
Tỷ lệ **cellular (63.5%)** cao hơn **telephone (36.5%)**.  
Điều này phản ánh đúng thực tế giai đoạn 2008–2010, khi ngân hàng chuyển dần sang gọi qua **di động** để tiếp cận khách hàng nhanh hơn.  

Khi phân tích hiệu quả chiến dịch, cần kiểm tra xem **tỷ lệ “yes”** giữa hai kênh này có khác biệt đáng kể không.  
Nếu di động có tỷ lệ thành công cao hơn, đó là tín hiệu cho việc **ưu tiên kênh liên hệ** này trong các chiến dịch sau.

---

#### 2. `month` — Tháng gọi
Các tháng có nhiều cuộc gọi nhất:
- **May (33%)**
- **July (17%)**
- **August (15%)**
- **June (13%)**

→ Chiếm hơn **75% tổng số cuộc gọi**.  
Có thể đây là **chiến dịch mùa hè**, khi ngân hàng đẩy mạnh huy động vốn hoặc tung sản phẩm mới.

Các tháng khác (Oct, Sep, Mar, Dec) có tỷ lệ rất nhỏ → thường là **off-campaign** hoặc **chiến dịch thử nghiệm**.  
Nên kiểm tra thêm xem **tỷ lệ “yes” có biến động theo mùa** không; ví dụ: tháng May gọi nhiều nhưng hiệu quả có thể không cao.

---

#### 3. `day_of_week` — Ngày gọi
Phân bố **khá đồng đều (khoảng 19–21%)** mỗi ngày.  
Điều này cho thấy ngân hàng triển khai chiến dịch đều trong tuần, **không tập trung riêng vào đầu hoặc cuối tuần**.

Đây là đặc điểm tốt, giúp **loại bỏ bias theo ngày** khi huấn luyện mô hình.  
Tuy nhiên, vẫn nên xem thử **thứ Sáu** có tỷ lệ “yes” cao hơn không — vì khách hàng cuối tuần có thể tâm lý thoải mái hơn.

---

#### 4. `y` — Biến mục tiêu (kết quả)
Dữ liệu **rất mất cân bằng**:  
- **Yes:** 11.3%  
- **No:** 88.7%

Đây là đặc trưng nổi tiếng của **Bank Marketing Dataset**.  
Hệ quả: nếu huấn luyện mô hình mà **không xử lý imbalance**, mô hình sẽ thiên về dự đoán “no”.

→ Ở bước mô hình hóa, cần **cân bằng lớp** bằng:
- `class_weighting`
- `SMOTE`
- hoặc `resampling`

để đảm bảo mô hình học được tín hiệu thực sự của nhóm “yes”.


In [ ]:
#Kiem tra tan suat phan bo cua cac bien so
num_cols = ['duration', 'campaign', 'pdays', 'previous']

df.select(num_cols).describe().show()

percentiles = df.approxQuantile(num_cols, [0.25, 0.5, 0.75, 0.9, 0.95, 0.99], 0.01)
print(percentiles, "\n")


for i, c in enumerate(num_cols):
    res = dict(zip(["25%", "50%", "75%", "90%", "95%", "99%"], percentiles[i]))
    print(f"Percentile cua {c}: ")
    print(res, "\n")

+-------+------------------+-----------------+-----------------+-------------------+
|summary|          duration|         campaign|            pdays|           previous|
+-------+------------------+-----------------+-----------------+-------------------+
|  count|             41188|            41188|            41188|              41188|
|   mean| 258.2850101971448|2.567592502670681|962.4754540157328|0.17296299893172767|
| stddev|259.27924883646455|2.770013542902331|186.9109073447414|0.49490107983928927|
|    min|                 0|                1|                0|                  0|
|    max|              4918|               56|              999|                  7|
+-------+------------------+-----------------+-----------------+-------------------+

[[102.0, 177.0, 312.0, 526.0, 702.0, 4918.0], [1.0, 2.0, 3.0, 5.0, 6.0, 56.0], [999.0, 999.0, 999.0, 999.0, 999.0, 999.0], [0.0, 0.0, 0.0, 1.0, 1.0, 7.0]] 

Percentile cua duration: 
{'25%': 102.0, '50%': 177.0, '75%': 312.0, '90%': 5

### Nhận xét phân tích các biến số

#### 1. `duration` — Thời lượng cuộc gọi
- Trung bình khoảng **258 giây**, độ lệch chuẩn gần bằng trung bình → **phân bố rất lệch phải (right-skewed)**.  
- **75%** cuộc gọi kéo dài ≤ **312 giây**, nhưng **1% trên cùng** lên tới gần **5000 giây (~82 phút)** → có **outlier mạnh**.  
- Điều này hợp lý, vì đa số khách hàng từ chối sớm; còn các cuộc gọi dài thường đến từ khách hàng có hứng thú hoặc nhân viên thuyết phục lâu.  
- Tuy nhiên, như đã nói ở phần trước, `duration` **không nên dùng để dự đoán trực tiếp**, vì đây là **kết quả sau khi gọi**, không phải thông tin biết trước.  
  → Dùng trong **EDA (phân tích khám phá dữ liệu)** để hiểu hành vi khách hàng là hợp lý.



#### 2. `campaign` — Số lần liên hệ trong chiến dịch hiện tại
- **Trung vị = 2**, **75% ≤ 3**, nhưng **1% cao nhất** lên tới **56 lần** → có những khách hàng bị gọi **hơn 50 lần (!)**  
- Đây là **outlier rõ ràng**, nhưng lại chứa **thông tin hành vi**: chiến dịch đã cố gắng “đuổi theo” khách hàng đó.  
- Khi mô hình hóa, nên:
  - **Giới hạn (clip)** giá trị tối đa ở một ngưỡng hợp lý (ví dụ 10), hoặc  
  - **Log-transform** để giảm ảnh hưởng của các giá trị cực lớn.



#### 3. `pdays` — Số ngày từ lần liên hệ trước
- Tất cả các percentile đều = **999**, cho thấy **phần lớn khách hàng chưa từng được liên hệ trước đây**.  
- Theo mô tả dữ liệu, **999 nghĩa là “no previous contact”**.  
- Biến này ít thông tin trực tiếp, nên khi xử lý có thể:
  - Tạo biến nhị phân mới: `has_contact_before = (pdays != 999)`, hoặc  
  - Giữ nguyên 999 và để mô hình tự học (tùy thuật toán).



#### 4. `previous` — Số lần liên hệ trước chiến dịch hiện tại
- **75% khách hàng chưa từng được liên hệ (0)**, **95% ≤ 1**, chỉ vài người đến **7 lần**.  
- Nghĩa là **đa số là khách hàng mới**.  
- Biến này có **phân bố rất lệch (one-sided)**, nên khi đưa vào mô hình cần:
  - **Chuẩn hóa (scaling)** hoặc  
  - **Binning (phân nhóm)** để mô hình dễ học và ổn định hơn.
---

### 1. Insight về hành vi liên hệ (kênh – thời điểm – kết quả):

#### 1.1 Về kênh liên hệ 
→ Mục tiêu: tìm hiểu kênh và thời điểm nào hiệu quả nhất.
**Câu hỏi khai thác:**

- Kênh cellular có giúp tăng tỷ lệ phản hồi tích cực không?

- Liệu telephone có thể bị loại bỏ hoặc giảm tần suất để tiết kiệm chi phí?

In [69]:
#Tỷ lệ khách hàng "yes" của từng kênh
cellular_total = df.filter(df['contact'] == 'cellular').count()
yes_cellular = df.filter((df['contact'] == 'cellular') & (df['y'] == 'yes')).count()
cellular_ratio = round(yes_cellular / cellular_total * 100, 2)
print(f"Ty le khách hang say yes trong cellular la:{cellular_ratio}% tren tong so {cellular_total} \n")



telephone_total = df.filter(df['contact'] == 'telephone').count()
yes_telephone = df.filter((df['contact'] == 'telephone') & (df['y'] == 'yes')).count()
telephone_ratio = round(yes_telephone / telephone_total * 100, 2)
print(f"ty le  khach hang say yes khi su dung telephone la:{telephone_ratio}% tren tong so {telephone_total} ")



Ty le khách hang say yes trong cellular la:14.74% tren tong so 26144 

ty le  khach hang say yes khi su dung telephone la:5.23% tren tong so 15044 


Phân tích cho thấy **kênh di động (cellular)** có tỷ lệ khách hàng đồng ý gửi tiết kiệm **cao gấp gần 3 lần** so với **điện thoại bàn (telephone)**.

- **Cellular:** 14.74% khách hàng đồng ý trên tổng **26,144 cuộc gọi**.  
- **Telephone:** 5.23% khách hàng đồng ý trên tổng **15,044 cuộc gọi**.

👉 **Kết luận:**  
Kênh **di động** cho thấy **hiệu quả vượt trội**, có thể do khách hàng **dễ tiếp cận hơn** và **phản hồi nhanh hơn**.  
**Đề xuất:** Ngân hàng nên **ưu tiên ngân sách và nhân sự cho kênh di động**, đồng thời **giảm tần suất gọi qua điện thoại bàn** để **tối ưu chi phí và nâng cao tỷ lệ chuyển đổi**.

---

#### 1.2 Về tháng gọi (month)

→ **Mục tiêu:** kiểm tra xem yếu tố **mùa vụ** có ảnh hưởng đến tỷ lệ khách hàng đồng ý (“yes”) hay không.

**Câu hỏi khai thác:**

- Có phải **tháng May** tuy có nhiều cuộc gọi nhưng **tỷ lệ thành công lại thấp**?

- **Tháng March** hoặc **September** có thể là “**thời điểm vàng**” – ít cuộc gọi nhưng **hiệu quả cao hơn**?

- Có **mối quan hệ nào giữa số lượng cuộc gọi và tỷ lệ thành công** (gọi nhiều chưa chắc tốt hơn)?


In [80]:
month_df = (
    df.groupBy('month')
      .agg(
          F.count('*').alias('total'),
          F.sum((F.col('y') == 'yes').cast('int')).alias('yes_count')
      )
      .withColumn('yes_ratio (%)', F.round(F.col('yes_count')/F.col('total')* 100, 2))
      .orderBy('yes_ratio (%)')
)
month_df.show()

+-----+-----+---------+-------------+
|month|total|yes_count|yes_ratio (%)|
+-----+-----+---------+-------------+
|  may|13769|      886|         6.43|
|  jul| 7174|      649|         9.05|
|  nov| 4101|      416|        10.14|
|  jun| 5318|      559|        10.51|
|  aug| 6178|      655|         10.6|
|  apr| 2632|      539|        20.48|
|  oct|  718|      315|        43.87|
|  sep|  570|      256|        44.91|
|  dec|  182|       89|         48.9|
|  mar|  546|      276|        50.55|
+-----+-----+---------+-------------+



Phân tích cho thấy các chiến dịch được triển khai mạnh nhất vào **tháng May, July và August**, nhưng **tỷ lệ thành công lại khá thấp**.  
Ngược lại, những tháng có **ít cuộc gọi** như **March, September, October và December** lại có tỷ lệ “yes” **rất cao** — thậm chí **gấp 5–8 lần so với tháng May**.

- **Tháng May:** 6.43% khách hàng đồng ý trên tổng **13,769 cuộc gọi**.  
- **Tháng August:** 10.6% khách hàng đồng ý trên tổng **6,178 cuộc gọi**.  
- **Tháng April:** 20.48% khách hàng đồng ý trên tổng **2,632 cuộc gọi**.  
- **Tháng March:** 50.55% khách hàng đồng ý trên tổng **546 cuộc gọi**.

👉 **Kết luận:**  
Hiệu quả chiến dịch có **tính mùa vụ rõ rệt**.  
Những tháng ngân hàng **gọi nhiều (đặc biệt là May, July)** lại có **tỷ lệ phản hồi thấp**,  
trong khi các tháng **ít gọi (March, September, October, December)** mang lại **tỷ lệ thành công vượt trội**.

**Đề xuất:**  
Ngân hàng nên **tái phân bổ lịch gọi**, **tăng cường chiến dịch** vào các tháng có hiệu quả cao,  
đồng thời **giảm tần suất** ở các tháng thấp hiệu quả như **May–July** để **nâng cao hiệu suất tổng thể**.

---

#### 1.3 Về ngày trong tuần (day_of_week)

→ **Mục tiêu:** xác định **ngày nào trong tuần** mang lại **tỷ lệ khách hàng đồng ý cao nhất**,  
từ đó hỗ trợ ngân hàng **lên lịch gọi tối ưu** cho đội ngũ tư vấn.

**Câu hỏi khai thác:**

- Liệu khách hàng có xu hướng **đồng ý nhiều hơn vào cuối tuần** (thứ Năm, thứ Sáu) khi **tâm lý thoải mái hơn**?

- Có **sự khác biệt rõ** giữa **đầu tuần** và **cuối tuần** hay không?

- Ngân hàng có thể **ưu tiên gọi vào các ngày “hiệu quả” hơn** để **tăng tỷ lệ chuyển đổi**?


In [93]:
dayOfWeek_df = (
    df.groupBy('day_of_week')
      .agg(
         F.count('*').alias('total'),
         F.sum((F.col('y') == 'yes').cast('int')).alias('yes_count')
      )
      .withColumn('yes_ratio(%)', F.round((F.col('yes_count')/F.col('total') * 100), 2))
      .orderBy('yes_ratio(%)',ascending = False)
)
dayOfWeek_df.show()

+-----------+-----+---------+------------+
|day_of_week|total|yes_count|yes_ratio(%)|
+-----------+-----+---------+------------+
|        thu| 8623|     1045|       12.12|
|        tue| 8090|      953|       11.78|
|        wed| 8134|      949|       11.67|
|        fri| 7827|      846|       10.81|
|        mon| 8514|      847|        9.95|
+-----------+-----+---------+------------+



Phân tích cho thấy **tỷ lệ khách hàng “say yes” cao nhất** rơi vào **thứ Năm (12.12%)**,  
tiếp theo là **thứ Ba (11.78%)** và **thứ Tư (11.67%)**.  
Trong khi đó, **thứ Hai có tỷ lệ thấp nhất (9.95%)** — tức là **đầu tuần khách hàng ít phản hồi tích cực hơn**.

- **Thứ Năm:** 12.12% khách hàng đồng ý trên tổng **8,623 cuộc gọi**.  
- **Thứ Ba:** 11.78% khách hàng đồng ý trên tổng **8,090 cuộc gọi**.  
- **Thứ Hai:** 9.95% khách hàng đồng ý trên tổng **8,514 cuộc gọi**.

👉 **Kết luận:**  
Tỷ lệ phản hồi tích cực có **xu hướng tăng dần về giữa và cuối tuần**, đạt **đỉnh vào thứ Năm**.  
Điều này gợi ý rằng **thời điểm giữa – cuối tuần** là **“khung giờ vàng” để triển khai cuộc gọi**,  
khi khách hàng có **tâm lý thoải mái** và **sẵn sàng tương tác hơn**.

---

### 2: “Tần suất & lịch sử liên hệ.”

#### 2.1. Tần suất gọi trong chiến dịch hiện tại – campaign

#### 2.2. Lịch sử liên hệ – pdays và has_contact_before


#### 2.3. Số lần liên hệ trong các chiến dịch trước – previous

In [46]:
contacted_before_df = df.filter(df['pdays'] != 999)
contacted_before_df.show(5)

print("--------------------------------------")
has_contact_bf_nums = contacted_before_df.count()
total = df.count()
ratio_of_contact = round(has_contact_bf_nums / total * 100, 2)

print("Tỷ lệ khách hàng đã từng được gọi trước đó (không phải lần đầu liên hệ):")
print(f"{ratio_of_contact:.2f}% trên tổng số {total} khách hàng.")


+---------+-----+-----------+--------+--------+-----+--------+---+
|  contact|month|day_of_week|duration|campaign|pdays|previous|  y|
+---------+-----+-----------+--------+--------+-----+--------+---+
|telephone|  nov|        wed|     119|       1|    6|       1| no|
| cellular|  nov|        mon|     112|       1|    4|       1| no|
| cellular|  nov|        mon|      94|       1|    4|       1| no|
| cellular|  nov|        mon|      77|       1|    3|       1| no|
| cellular|  nov|        mon|     200|       2|    4|       1| no|
+---------+-----+-----------+--------+--------+-----+--------+---+
only showing top 5 rows

--------------------------------------
Tỷ lệ khách hàng đã từng được gọi trước đó (không phải lần đầu liên hệ):
3.68% trên tổng số 41188 khách hàng.


### Nhận xét ngắn gọn

Chỉ **3.68%** khách hàng từng được gọi trước đó, nghĩa là **hơn 96%** là lần đầu tiên được liên hệ.  

Điều này xác nhận chiến dịch marketing của ngân hàng chủ yếu là **cold call**, không phải **chăm sóc khách hàng cũ**.  

Nhóm khách hàng “được gọi lại” tuy nhỏ, nhưng **có thể là nhóm tiềm năng cao hơn** — nên ở phần sau, bạn nên kiểm tra xem họ có **tỷ lệ “yes” cao hơn** hay không.  
Đó là một **insight quan trọng** để mô hình hóa hành vi **follow-up** trong chiến dịch.
